In [2]:
import pandas as pd

In [8]:
def split_tables(file_path):
    # Read the Excel file and select the 'All Cycles' sheet, starting from row 11 as the data
    df = pd.read_excel(file_path, sheet_name='Range 1', header=None, skiprows=10)

    # Get the unique strings from the first row
    unique_strings = df.iloc[0].unique()

    # Exclude specific strings from unique_strings
    excluded_strings = ['Well\nRow', 'Well\nCol', 'Content']
    unique_strings = [string for string in unique_strings if string not in excluded_strings]

    # Create separate DataFrames based on unique strings
    table_names=[]
    for string in unique_strings:
        columns = excluded_strings + [string]
        sub_df = df.loc[:, df.iloc[0].isin(columns)]
        sub_df.columns = list(sub_df.iloc[0].values[:3])+list(sub_df.iloc[1].values[3:])  # Assign first row as column names for columns 0, 1, 2 and keep the remaining column names
        sub_df = sub_df.iloc[2:]  # Remove the first two rows

        # Convert time string columns to minutes
       # time_columns = sub_df.columns[3:]

        #for time_column in time_columns:
         #   time_list = time_column.replace("min","").replace(" ","").split("h")
          #  hours=int(time_list[0])
           # if len(time_list[1])==0:
            #    minutes = 0
            #else:
             #   minutes = int(time_list[1])
            #timepoint = hours*60 + minutes
            #sub_df = sub_df.rename(columns={time_column: timepoint})

        for excl_string in excluded_strings:
            sub_df = sub_df.rename(columns={excl_string: excl_string.replace("\n","_").lower()})

        # save to a variable
        df_name=string.replace("(","").replace(")","").replace(" ","_").lower().replace("blank_corrected_based_on_raw_data","corrected")
        sub_df=sub_df.dropna(thresh=5)

        globals()[df_name] = sub_df.reset_index(drop=True)
#         print(df_name)
        table_names.append(df_name)
    return table_names


In [9]:
# Specify the file path of the Excel file
file_path = r'C:\Users\s1814897\Projects\BMGPlateReaderPython\example_inputs\3_11_2023.xlsx'
sheet_name = "All Cycles"

In [10]:
table_names = split_tables(file_path, sheet_name)
table_names

AttributeError: 'float' object has no attribute 'replace'

In [15]:
[name for name in table_names if "corrected" in name]

NameError: name 'table_names' is not defined

## Only work with corrected df

In [13]:
# Rename the samples according to excel sheet values
rename_df = pd.read_excel(file_path, sheet_name='SampleNames')
keys = rename_df.RenameFrom.tolist()
values = rename_df["RenameTo"].tolist()
rename_dict = dict(zip(keys, values))
corrected_table_names=[name for name in table_names if "corrected" in name]
for table_name in corrected_table_names:
    for row in range(len(globals()[table_name])):
        globals()[table_name].loc[row,"content"] = rename_dict[globals()[table_name].loc[row,"content"]]

In [14]:

corrected_600_1

,well_row,well_col,content,0,30,60,90,120,150,180,...,900,930,960,990,1020,1050,1080,1110,1140,1170
0,A,1,dad,0.04,0.054,0.105,0.146,0.179,0.239,0.305,...,0.791,0.872,0.805,0.791,0.847,0.706,0.851,0.59,0.829,0.652
1,A,2,dad,0.049,0.046,0.091,0.119,0.162,0.217,0.281,...,0.836,0.872,0.811,0.804,0.866,0.725,0.867,0.61,0.845,0.674
2,A,3,dad,0.04,0.042,0.081,0.126,0.165,0.219,0.293,...,0.808,0.873,0.814,0.807,0.865,0.726,0.873,0.611,0.849,0.678
3,B,1,asf,0.041,0.044,0.077,0.103,0.136,0.196,0.23,...,0.808,0.873,0.822,0.807,0.86,0.727,0.879,0.618,0.856,0.675
4,B,2,asf,0.041,0.042,0.076,0.106,0.137,0.195,0.233,...,0.861,0.923,0.871,0.849,0.9,0.784,0.928,0.664,0.901,0.718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,C,11,fads,0.044,0.045,0.059,0.074,0.11,0.162,0.242,...,0.816,0.87,0.807,0.79,0.855,0.705,0.852,0.59,0.826,0.655
68,C,12,fads,0.048,0.049,0.055,0.063,0.079,0.101,0.15,...,0.726,0.787,0.722,0.71,0.771,0.624,0.77,0.509,0.744,0.571
69,D,10,fads,0.043,0.043,0.052,0.055,0.059,0.062,0.059,...,0.013,0.077,0.02,0.014,0.073,-0.067,0.078,-0.184,0.055,-0.121
70,D,11,fads,0.042,0.051,0.051,0.055,0.063,0.061,0.063,...,0.021,0.084,0.026,0.018,0.08,-0.061,0.084,-0.178,0.061,-0.112


In [24]:
for name in corrected_table_names:
    globals()[f"transposed_{name}"]=globals()[name].T
    globals()[f"transposed_{name}"]=globals()[f"transposed_{name}"][2:]
    print(f"transposed_{name}")

transposed_corrected_600_1
transposed_corrected_485_2
transposed_corrected_584_3


In [49]:
#This is to join two dataframes for analysis in Graphpad. 

# Split the columns of both dataframes into groups of 3
columns_transposed_corrected_600_1 = [transposed_corrected_600_1.columns[i:i+3] for i in range(0, len(transposed_corrected_600_1.columns), 3)]
columns_transposed_corrected_485_2 = [transposed_corrected_485_2.columns[i:i+3] for i in range(0, len(transposed_corrected_485_2.columns), 3)]
columns_transposed_corrected_584_3 = [transposed_corrected_584_3.columns[i:i+3] for i in range(0, len(transposed_corrected_584_3.columns), 3)]
# Create a list to hold the concatenated dataframes
concatenated_dfs = []

# Iterate over the column groups and concatenate the corresponding columns
for cols1, cols2 in zip(columns_transposed_corrected_600_1, columns_transposed_corrected_485_2):
    concatenated_df = pd.concat([transposed_corrected_600_1[cols1], transposed_corrected_485_2[cols2]], axis=1)
    concatenated_dfs.append(concatenated_df)

# Concatenate the resulting dataframes vertically
final_df = pd.concat(concatenated_dfs, axis=1)
sfGFP=final_df.reset_index()
sfGFP.to_excel("sfGFP.xlsx", index=False)

###### 
# Now for the 2nd dataframe
#####
concatenated_dfs = []
for cols1, cols2 in zip(columns_transposed_corrected_600_1, columns_transposed_corrected_584_3):
    concatenated_df = pd.concat([transposed_corrected_600_1[cols1], transposed_corrected_584_3[cols2]], axis=1)
    concatenated_dfs.append(concatenated_df)

# Concatenate the resulting dataframes vertically
final_df = pd.concat(concatenated_dfs, axis=1)
mCherry=final_df.reset_index()
mCherry.to_excel("mCherry.xlsx", index=False)

